# 지리 시각화 코드, folium을 사용

### 생활인구 데이터 홈페이지
https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do

In [2]:
from selenium import webdriver
import time
import folium
import os
import json
import pandas as pd
import numpy as np
import datetime

# 전처리 된 데이터를 불러오는 과정

In [4]:
data_path = '/home/zimin/datasets/seoul/'
analysis = datetime.datetime(2019,11, 23)
week = {0 : '월요일', 1 : '화요일', 2 : '수요일', 3 : '목요일', 4 : '금요일', 5 : '토요일', 6 : '일요일'}
weekday = week[analysis.weekday()]
date = str(analysis.year) + str(analysis.month) + str(analysis.day) 

mapping = pd.read_csv(os.path.join(data_path,'행정동코드_매핑정보_2018.csv'))
mapping.drop(0, axis =0, inplace = True)

district = mapping['시군구명'].unique().tolist()

dtos = {}
for i in district:
    dtos[i] = mapping[mapping['시군구명']==i]['행정동명'].values.tolist()

data_path = '/home/zimin/datasets/seoul/'
geo_path = data_path + 'skorea_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str['features'] = [i for i in geo_str['features'] if i['properties']['code'][:2] == '11']

inner = pd.read_csv(os.path.join(data_path, 'pre_INNER_PEOPLE_' + date + '.csv'), index_col=0)

# 시간, 분석 나이대, 장소를 지정

In [5]:
when = 15
who = ['115','215']
if who[0] != '300':
    whose = who[0][1:] + '~' + str(eval(who[-1][1:] )+5) + '세'
else:
    whose = '외국인'
where =['갈현1동', '대치2동']

In [6]:
check = inner[(inner['시간대구분'] == when) & (inner['행정동이름'].isin(where))][['행정동이름','행정동코드','자치구이름'] + who]

In [7]:
check['자치구코드'] = 0

for i in district:
    df = check[check['자치구이름'] == i]
    for j in dtos[i]:
        df['자치구코드'] = j
        check = check.append(df)
        

check = check[check['자치구코드'] != 0]

/home/zimin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
check['자치구코드'] = check['자치구코드'].apply(lambda x : inner[inner['행정동이름']==x]['행정동코드'].iloc[0])
df = check[check['자치구이름']=='강남구'].iloc[:1,:]
df['자치구코드'] = '1123051'
check = check.append(df)

In [9]:
check[who] = check[who].replace('*',0)
check[who] = check[who].astype(int)
check['sum'] = check[who].sum(axis=1)
check['행정동코드'] = check['행정동코드'].apply(str)

In [10]:
gal = check[check['행정동이름']=='갈현1동']
gal['sum'] = gal['sum'].apply(lambda x: np.power(x, 1/2))
gal['자치구코드'] = gal['자치구코드'].apply(str)

/home/zimin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/zimin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
daechi = check[check['행정동이름']=='대치2동']
daechi['sum'] = daechi['sum'].apply(lambda x: np.power(x, 1/3))
daechi['자치구코드'] = daechi['자치구코드'].apply(str)

/home/zimin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/zimin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# folium을 이용해 지리 시각화  : output이 html로 떨어짐

## html 스크린샷을 찍기 위해 selenium을 사용, chromedriver 설치 필요

In [12]:
fn= date + weekday + ' ' +str(when)+ '시 ' + whose  + '.html'

seoul_map = folium.Map(location =[37.5502, 126.982], zoom_start=11, tiles = 'cartodbpositron')
seoul_map.choropleth(geo_data = geo_str,
                    data = gal,
                    columns = ['자치구코드', 'sum'],
                    fill_color = 'Reds',
                    bins = 9,
                    key_on = 'properties.code',
                    highlight = True,
                    fill_opacity = 0.7, line_opacity = 1,
                    legend_name = 'After power 1/3')

title_html = '''
             <h3 align="center" style="font-size:20px"><b>{}</b></h3>
             '''.format(fn[:-5])
seoul_map.get_root().html.add_child(folium.Element(title_html))

delay=1
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd() + '/html_data',mapfile=fn)
seoul_map.save('./html_data/' + fn)
browser = webdriver.Chrome('/usr/bin/chromedriver')
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('./screenshot/' + fn[:-5] + '.png')
browser.quit()

/home/zimin/anaconda3/lib/python3.7/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
